In [2]:
import itertools
import os

import matplotlib.pylab as plt
import numpy as np
import time
import pandas as pd



from absl import app
from absl import flags
import tensorflow as tf


import tensorflow as tf
import tensorflow_hub as hub

import sklearn.metrics
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
import seaborn as sns

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)

TF version: 2.6.0
Hub version: 0.8.0


In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
  tf.config.experimental.set_memory_growth(device, True)

print('Num Gpu Avaible' ,len(physical_devices))

Num Gpu Avaible 1


In [4]:
tf.test.is_built_with_cuda

<function tensorflow.python.platform.test.is_built_with_cuda()>

In [5]:
import sys
sys.version

'3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]'

In [6]:
batch_size = 10
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  "classification/ip102_v1.1/train",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  "classification/ip102_v1.1/val",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
    "classification/ip102_v1.1/test",
    labels='inferred',
    label_mode='categorical',
    image_size=(img_height, img_width),
    batch_size=batch_size
 )

Found 45095 files belonging to 102 classes.
Found 7508 files belonging to 102 classes.
Found 22619 files belonging to 102 classes.


In [6]:
import pathlib 
import numpy as np

data_dir = pathlib.Path("classification/ip102_v1.1/train")
class_names = np.array(sorted([item.name for item in data_dir.glob("*")])) # creating a list of class names from subdirectory 
print(class_names)

['Adristyrannus' 'Aleurocanthus spiniferus' 'Ampelophaga'
 'Aphis citricola Vander Goot' 'Apolygus lucorum' 'Bactrocera tsuneonis'
 'Beet spot flies' 'Brevipoalpus lewisi McGregor' 'Ceroplastes rubens'
 'Chlumetia transversa' 'Chrysomphalus aonidum' 'Cicadella viridis'
 'Cicadellidae' 'Colomerus vitis' 'Dacus dorsalis(Hendel)' 'Dasineura sp'
 'Deporaus marginatus Pascoe' 'Erythroneura apicalis'
 'Icerya purchasi Maskell' 'Lawana imitata Melichar' 'Limacodidae'
 'Locustoidea' 'Lycorma delicatula' 'Mango flat beak leafhopper' 'Miridae'
 'Nipaecoccus vastalor' 'Panonchus citri McGregor' 'Papilio xuthus'
 'Parlatoria zizyphus Lucus' 'Phyllocnistis citrella Stainton'
 'Phyllocoptes oleiverus ashmead' 'Pieris canidia'
 'Polyphagotars onemus latus' 'Potosiabre vitarsis' 'Prodenia litura'
 'Pseudococcus comstocki Kuwana' 'Rhytidodera bowrinii white'
 'Rice Stemfly' 'Salurnis marginella Guerr' 'Scirtothrips dorsalis Hood'
 'Sternochetus frigidus' 'Tetradacus c Bactrocera minax' 'Thrips'
 'Toxop

In [7]:
class_names = tuple(val_ds.class_names)
print(class_names)


('Adristyrannus', 'Aleurocanthus spiniferus', 'Ampelophaga', 'Aphis citricola Vander Goot', 'Apolygus lucorum', 'Bactrocera tsuneonis', 'Beet spot flies', 'Brevipoalpus lewisi McGregor', 'Ceroplastes rubens', 'Chlumetia transversa', 'Chrysomphalus aonidum', 'Cicadella viridis', 'Cicadellidae', 'Colomerus vitis', 'Dacus dorsalis(Hendel)', 'Dasineura sp', 'Deporaus marginatus Pascoe', 'Erythroneura apicalis', 'Icerya purchasi Maskell', 'Lawana imitata Melichar', 'Limacodidae', 'Locustoidea', 'Lycorma delicatula', 'Mango flat beak leafhopper', 'Miridae', 'Nipaecoccus vastalor', 'Panonchus citri McGregor', 'Papilio xuthus', 'Parlatoria zizyphus Lucus', 'Phyllocnistis citrella Stainton', 'Phyllocoptes oleiverus ashmead', 'Pieris canidia', 'Polyphagotars onemus latus', 'Potosiabre vitarsis', 'Prodenia litura', 'Pseudococcus comstocki Kuwana', 'Rhytidodera bowrinii white', 'Rice Stemfly', 'Salurnis marginella Guerr', 'Scirtothrips dorsalis Hood', 'Sternochetus frigidus', 'Tetradacus c Bactroc

In [7]:
mobilenet_v3 ="https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5"

feature_extractor_model  = mobilenet_v3

In [8]:
do_fine_tuning = True 

In [9]:
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=do_fine_tuning)

In [10]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

In [11]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [12]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(10, 224, 224, 3)
(10,)


In [13]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

(10, 1280)


In [18]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dropout(rate=0.2),
  tf.keras.layers.Dense(num_classes,activation='softmax')
])


model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              4226432   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 102)               130662    
Total params: 4,357,094
Trainable params: 4,332,694
Non-trainable params: 24,400
_________________________________________________________________


In [19]:
predictions = model(image_batch)

In [20]:
predictions.shape

TensorShape([10, 102])

In [21]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate= 0.001),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
  metrics=['accuracy'])


In [18]:
NUM_EPOCHS = 10

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS,
                   )

Epoch 1/10
 797/1410 [===============>..............] - ETA: 1:57 - loss: 2.6783 - accuracy: 0.4175 - ETA: 4:01 - loss: 3.5136 - accuracy:  - ETA: 4:01 - los - ETA: 2:53 - los - ETA: 2:11 - - ETA: 2:08 - loss: 2.7117  - ETA: 2:06 - loss: 2.7086 - accuracy: 0.

KeyboardInterrupt: 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(NUM_EPOCHS)

plt.figure(figsize=(15,10))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(train_ds)

In [ ]:
predicted_batch = model.predict(image_batch)
predicted_id = tf.math.argmax(predicted_batch, axis=1)
predicted_label_batch = class_names[predicted_id]
print(predicted_label_batch)

In [ ]:
plt.figure(figsize=(15,15))
plt.subplots_adjust(hspace=0.5)

for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(image_batch[n])
  plt.title(predicted_label_batch[n].title())
  plt.axis('off')
_ = plt.suptitle("Model predictions")

In [ ]:
predictions = np.array([])
labels =  np.array([])
for x, y in test_ds:
  predict_x=model.predict(x) 
  classes_x=np.argmax(predict_x,axis=1) 
  predictions = np.concatenate([predictions, classes_x])
  labels = np.concatenate([labels, np.argmax(y.numpy(), axis=-1)])


tf.math.confusion_matrix(labels=labels, predictions=predictions).numpy()

In [ ]:
cf_matrix = confusion_matrix(labels, predictions, normalize='true')
plt.figure(figsize = (40,40))
sns.heatmap(cf_matrix,
            annot=True,
            xticklabels = sorted(set(class_names)),
            yticklabels = sorted(set(class_names)),
            )
plt.xlabel("Predicted Classes",fontsize=15)
plt.ylabel("True Classes",fontsize=15)
plt.title('Normalized Confusion Matrix')
plt.show()

In [ ]:
print(classification_report(labels, predictions,target_names=class_names))
